In [1]:
from utils import *

In [2]:
# Warning: 1.5GB file! Make sure you have enough RAM (~3GB) to open.
new_names = {
    'Vehicle#': 'vehicle_id',
    'District': 'district',
    'Column1.visit_date': 'visit_date',
    'Column1.visit_date.date.time': 'visit_timestamp',
    'Column1.control_group_data': 'control_group',
    'Column1.workzone_id': 'workzone_id',
    'Column1.cause': 'cause',
    'Column1.message': 'message',
    'Column1.points.features.geometry.coordinates': 'coordinates',
    'Column1.points.features.properties.speed': 'speed',
    'Column1.points.features.properties.bearing': 'bearing',
    'Column1.points.features.properties.timestamp': 'event_time',
    'Column1.points.features.properties.timestamp.date.time': 'event_timestamp',
    'Column1.points.features.properties.accel': 'acceleration',
    'Column1.points.features.properties.brake_avg': 'brake_average',
    'Column1.points.features.properties.brake_max': 'brake_maximum',
    'Column1.points.features.properties.brake_start': 'brake_start',
    'Column1.points.features.properties.brake_stop': 'brake_stop'
}

# Rename the columns
df = df.rename(columns=new_names)
df = pd.read_csv('data.csv')
df

,Vehicle#,District,Column1.visit_date,Column1.visit_date.date.time,...,Column1.points.features.properties.brake_avg,Column1.points.features.properties.brake_max,Column1.points.features.properties.brake_start,Column1.points.features.properties.brake_stop
0,210821.0,11.0,2024-11-23T00:00:33,1.40e+10,...,NaN,NaN,NaN,NaN
1,210821.0,11.0,2024-11-23T00:00:33,1.40e+10,...,NaN,NaN,NaN,NaN
2,210821.0,11.0,2024-11-23T00:00:33,1.40e+10,...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9665540,251592.0,11.0,2024-12-23T23:59:45,1.40e+10,...,NaN,NaN,NaN,NaN
9665541,251592.0,11.0,2024-12-23T23:59:45,1.40e+10,...,NaN,NaN,NaN,NaN
9665542,251592.0,11.0,2024-12-23T23:59:45,1.40e+10,...,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['Vehicle#', 'District', 'Column1.visit_date',
       'Column1.visit_date.date.time', 'Column1.control_group_data',
       'Column1.workzone_id', 'Column1.cause', 'Column1.message',
       'Column1.points.features.geometry.coordinates',
       'Column1.points.features.properties.speed',
       'Column1.points.features.properties.bearing',
       'Column1.points.features.properties.timestamp',
       'Column1.points.features.properties.timestamp.date.time',
       'Column1.points.features.properties.accel',
       'Column1.points.features.properties.brake_avg',
       'Column1.points.features.properties.brake_max',
       'Column1.points.features.properties.brake_start',
       'Column1.points.features.properties.brake_stop'],
      dtype='object')